In [ ]:
import pyspark.sql.types as st
import pyspark.sql.functions as sf
from pyspark.sql import SparkSession, SQLContext

In [ ]:
# Spark Session app
spark = SparkSession \
  .builder \
  .appName('birdie') \
  .getOrCreate()

# Create a SQL Context
sc = spark.sparkContext

In [ ]:
path_principal = "/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/enwiki_latest_pages_articles_files_new/*"
path_category = "/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/enwiki-category-sql/category.csv"

In [ ]:
SCHEMA = spark.read.json(path_principal).schema

In [ ]:
df_original = spark \
    .read \
    .json(path_principal, SCHEMA)

#### Start Category Validate.

In [ ]:
read_category = spark \
    .read.option("delimiter","|") \
    .csv(path_category, header=True)

In [ ]:
# Romove equals duplicity registry in dataset
# Create and remove max field
read_category_distinct = read_category \
    .groupby("cat_id","cat_title","cat_pages","cat_subcats","cat_files") \
    .agg(sf.max("cat_id").alias("max")).drop("max")

In [ ]:
# Show if the dataset have still duplicitys
read_category_distinct\
    .groupby("cat_id")\
    .count()\
    .filter("count > 1")\
    .show()

In [ ]:
read_category_distinct.filter("cat_id == 12").show(truncate=False)

In [ ]:
read_category_distinct.count()

In [ ]:
df_original.filter("id == 12").show()

In [ ]:
df_union_dataset = df_original \
    .join(read_category_distinct, df_original.id == read_category_distinct.cat_id, how='left')

In [ ]:
df_union_dataset.filter("cat_id != ''").show(1)

In [ ]:
df_union_dataset.show(1)

In [ ]:
df_union_dataset \
    .write.mode("overwrite") \
    .json("/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final")

## ELSK + Python

In [ ]:
!pip install elasticsearch

In [ ]:
# Import Elasticsearch package 
from elasticsearch import Elasticsearch, helpers
import os, json

In [ ]:
# Connect to the elastic cluster
es = Elasticsearch([{'host':'172.19.0.3','port':9200}])

In [ ]:
# test connection, if True its ok!
es.ping()

In [ ]:
# Path with all json files.
directory = "/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/"

# while for the recover all the json files and broken each in individual payload per id
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        # recover all the json files
        with open(directory+filename, 'r') as open_file:
            print(directory+filename)
            # open the json files and broken each part in one
            data = [json.loads(line) for line in open(directory+filename).readlines()]
            
            # Index payload per payload in ELSK.
            helpers.bulk(es, data, index='birdie-wiki-final', doc_type='articles', request_timeout=40)


#### Search with API

In [ ]:
res = es.search(index='birdie-wiki-final',body={'query':{'match_all':{}}})

print('Got %d hits:' %res['hits']['total'])

In [ ]:
res = es.search(index='birdie-wiki-final',body={'query':{}})

print(res['hits']['hits'])

In [ ]:
res = es.search(index='birdie-wiki-final',body={'query':{'match':{'id': 13809905}}})

print(res['hits']['hits'])

## Endpoints

In [ ]:
import requests

In [ ]:
resp = requests.get('https://localhost:8080/find')

In [ ]:
if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))
    
for todo_item in resp.json():
    print('{} {}'.format(todo_item['id'], todo_item['summary']))

In [ ]:
#let's iterate over swapi people documents and index them
import json

In [ ]:
#let's iterate over swapi people documents and index them

r = requests.get('http://localhost:9200') 

i = 1

# while r.status_code == 200:
r = requests.get('http://swapi.co/api/people/'+ str(i))

es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    #i=i+1
 
print(i)